In [3]:
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import nltk
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
import tensorflow as tf
# from tensorflow.compat.v1.contrib import rnn
import csv
import reader

In [4]:
file_name = 'select_description_from_sys_district.csv'
if reader.check_file(file_name, url=''):
    hotelDescs = reader.read_data_as_text(file_name)
else:
    quit()

文件  select_description_from_sys_district.csv 存在。盛宴大幕拉开了。


In [7]:
def build_dataset(documents):
    chars = []
    # 这是一些列表．外部列表表示每个文档,内部列表表示给定文档中的单词     
    data_list = []
  
    for d in documents:
        chars.extend(d)
    print('找到 %d 个单词.'%len(chars))
    count = []
    # 根据频率对bigram进行排序（最高的排在第一位）
    count.extend(collections.Counter(chars).most_common())
    
    # 通过给出字典的当前长度为每个bigram创建一个ID
    # 并将该元素添加到字典中
    # 从分配给文本中稀有单词的“UNK”开始    
    dictionary = dict({'UNK':0})
    for char, c in count:
        # 如果统计的频率超过10，则仅增加一个bigram到字典中
        if c > 10:
            dictionary[char] = len(dictionary)    
    
    unk_count = 0
    # 遍历所有文本,我们将每个字符串形式的单词替换为单词的ID     
    for d in documents:
        data = list()
        for char in d:
            # 如果一个单词存在于词典中,则使用该单词ID,否则使用特殊标记“UNK”的ID            
            if char in dictionary:
                index = dictionary[char]        
            else:
                index = dictionary['UNK']
                unk_count += 1
            data.append(index)
            
        data_list.append(data)
        
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data_list, count, dictionary, reverse_dictionary

global data_list, count, dictionary, reverse_dictionary,vocabulary_size

# 打印数据一些的统计信息
data_list, count, dictionary, reverse_dictionary = build_dataset(hotelDescs)
print('常用的词 (+UNK)', count[:5])
print('不常用的词 (+UNK)', count[-15:])
print('样本数据', data_list[0][:10])
print('样本数据', data_list[1][:10])
print('词汇量: ',len(dictionary))
vocabulary_size = len(dictionary)
#del hotelDescs  # 减少内存

找到 894831 个单词.
常用的词 (+UNK) [('，', 33827), ('、', 22474), ('。', 17855), ('的', 16469), ('店', 14173)]
不常用的词 (+UNK) [('棒', 1), ('瓢', 1), ('俚', 1), ('狼', 1), ('猬', 1), ('涨', 1), ('℃', 1), ('催', 1), ('厘', 1), ('逢', 1), ('庠', 1), ('剥', 1), ('｡', 1), ('辨', 1), ('蔼', 1)]
样本数据 [11]
样本数据 [6]
词汇量:  1764
